In [1]:
import numpy as np
import pandas as pd

from rectools import Columns

Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

**Итоговая задача** - построить схему валидации для данного соревнования с учетом особенностей сорвенования
- Между `train` и `test` не должно быть общих пользователей
- Количество фолдов задается через параметр класса `n_folds`
- В `test` должно быть не более `p` последних треков (параметр класса `p`)

In [2]:
data = pd.read_csv('../../PycharmProjects/RecSys_DEMO/Source/Yandex/train',
                   header=None,
                   names=['tracks'])

data['tracks'] = data.tracks.apply(lambda x: x.split()[::-1])
data = data.explode('tracks').reset_index()

data.rename(
    columns={
        'index': Columns.User,
        'tracks': Columns.Item,
    },
    inplace=True)

data['order'] = data.groupby(Columns.User).cumcount() + 1

data

,user_id,item_id,order
0,0,388242,1
1,0,278503,2
2,0,102795,3
3,0,470957,4
4,0,159637,5
...,...,...,...
94188629,1160083,19120,252
94188630,1160083,326821,253
94188631,1160083,214132,254
94188632,1160083,352098,255


In [3]:
# !mkdir Source
# data.to_csv('./Source/dataframe.csv', index=False)

In [4]:
# data = pd.read_csv('./Source/dataframe.csv')
# data.head()

In [9]:
class UsersKFoldPOut():
    def __init__(self, n_folds, p, user_column=Columns.User, random_seed=23):
        self.n_folds = n_folds
        self.user_column = user_column
        self.random_seed = random_seed
        self.p = p
    
    def split(self, df):
        users = df[self.user_column].unique()
        users_count = len(users)
        
        np.random.seed(self.random_seed)
        np.random.shuffle(users)
        
        fold_sizes = np.full(self.n_folds, users_count // self.n_folds, dtype=int)
        fold_sizes[: users_count % self.n_folds] += 1
        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]
            test_mask = df[self.user_column].isin(test_fold_users)
            train_mask = ~test_mask
            current = stop
            yield train_mask, test_mask & (df.order <= self.p)

In [10]:
cv = UsersKFoldPOut(n_folds=3, p=1)

for i, (train_mask, test_mask) in enumerate(cv.split(data)):
    train = data[train_mask]
    test = data[test_mask]
    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')
    print(f'Пересекающиеся пользователи train и test - {np.intersect1d(train.user_id.unique(), test.user_id.unique())}')
    print(f'Максимальное количесво треков у пользователей в test = {test.order.max()}')

Fold#0 | Train: 62769950, Test: 386695
Пересекающиеся пользователи train и test - []
Максимальное количесво треков у пользователей в test = 1
Fold#1 | Train: 62749697, Test: 386695
Пересекающиеся пользователи train и test - []
Максимальное количесво треков у пользователей в test = 1
Fold#2 | Train: 62857621, Test: 386694
Пересекающиеся пользователи train и test - []
Максимальное количесво треков у пользователей в test = 1


In [11]:
cv = UsersKFoldPOut(n_folds=3, p=2)

for i, (train_mask, test_mask) in enumerate(cv.split(data)):
    train = data[train_mask]
    test = data[test_mask]
    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')
    print(f'Пересекающиеся пользователи train и test - {np.intersect1d(train.user_id.unique(), test.user_id.unique())}')
    print(f'Максимальное количесво треков у пользователей в test = {test.order.max()}')

Fold#0 | Train: 62769950, Test: 773390
Пересекающиеся пользователи train и test - []
Максимальное количесво треков у пользователей в test = 2
Fold#1 | Train: 62749697, Test: 773390
Пересекающиеся пользователи train и test - []
Максимальное количесво треков у пользователей в test = 2
Fold#2 | Train: 62857621, Test: 773388
Пересекающиеся пользователи train и test - []
Максимальное количесво треков у пользователей в test = 2
